In [1]:
import tensorflow as tf
from sentence_transformers import SentenceTransformer
import json
import numpy as np
import requests
from components.response import retrieve_context, generate_response, embed_question
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

d:\CSE 453 - Tensorflow and other tools in Healthcare\Medical Chatbot Deployment\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = T5Tokenizer.from_pretrained("original_model")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
retriever_model = SentenceTransformer("all-MiniLM-L6-v2")

In [4]:
with open("data/embeddings/corpus.json", "r") as f:
    corpus = json.load(f)

In [5]:
def generate_response(question, retrieved_context):
    # Prepare the input
    input_text = f"Context: {retrieved_context} Question: {question}"

    # Tokenize the input text
    encoding = tokenizer(
        input_text, return_tensors="tf", padding=True, truncation=True, max_length=512
    )

    # Convert to lists for JSON serialization
    input_ids = encoding["input_ids"].numpy().tolist()
    attention_mask = encoding["attention_mask"].numpy().tolist()

    # For T5, decoder_input_ids can be the same as input_ids for inference
    decoder_input_ids = input_ids  # This is typically the same for inference
    decoder_attention_mask = attention_mask  # This is typically the same for inference

    # Prepare the payload for TensorFlow Serving
    payload = {
        "signature_name": "serving_default",
        "instances": [
            {
                "input_ids": input_ids[0],  # Use the first (and only) input
                "attention_mask": attention_mask[0],  # Include attention mask
                "decoder_input_ids": decoder_input_ids[0],  # Include decoder input ids
                "decoder_attention_mask": decoder_attention_mask[
                    0
                ],  # Include decoder attention mask
            }
        ],
    }

    # Send the request to TensorFlow Serving
    url = "http://localhost:8501/v1/models/fine_tuned_t5:predict"
    response = requests.post(url, json=payload)

    # Check for response errors
    if response.status_code != 200:
        raise Exception(
            f"Request failed with status code {response.status_code}: {response.text}"
        )

    # Get the output from the response
    output = response.json()

    # Print all keys in the output
    print("Response Keys:", output.keys())  # Print the keys in the response

    # Extract the predictions
    if "predictions" in output:
        predictions = output["predictions"]
        print("Predictions Keys:")

        # Iterate over each prediction and print its keys
        for i, prediction in enumerate(predictions):
            print(f"Keys in prediction {i}:", prediction.keys())

        # Assuming we want to decode the first prediction
        if predictions:
            first_prediction = predictions[0]

            # Check if 'logits' is present
            if "logits" in first_prediction:
                logits = first_prediction["logits"]

                # Get the top-k predicted token IDs for each position
                top_k = 5  # Change this to the number of responses you want
                top_k_indices = np.argsort(logits, axis=-1)[
                    :, -top_k:
                ]  # Get the indices of the top-k logits

                # Decode the top-k predicted token IDs into text
                all_responses = []
                for k in range(top_k):
                    predicted_ids = top_k_indices[:, k]  # Get the k-th top prediction
                    response_text = tokenizer.decode(
                        predicted_ids[0], skip_special_tokens=True
                    )
                    all_responses.append(response_text)

                # Print all generated responses
                print("Generated Responses:")
                for idx, response in enumerate(all_responses):
                    print(f"Response {idx + 1}: {response}")
            else:
                raise ValueError("Expected 'logits' key not found in predictions.")
    else:
        raise ValueError("Unexpected output format: 'predictions' key not found.")

In [7]:
def rag_pipeline(question, corpus, k=3):
    # ncode the question
    question_embedding = embed_question(question, retriever_model).astype("float32")

    # Retrieve top-k contexts
    indices, distances = retrieve_context(question_embedding=question_embedding, k=k)
    retrieved_context = " ".join([corpus[i] for i in indices[0]])

    print(f"Retrieved Context: {retrieved_context}")

    generate_response(question=question, retrieved_context=retrieved_context)

In [8]:
if __name__ == "__main__":
    question = "Stomach Issue Symptoms?"

    rag_pipeline(question=question, corpus=corpus)

Distances: [[0.5678699 0.5689838 0.5742972]]
Indices: [[100444  92174  66184]]
Retrieved Context: Hello.. I feel lower abdominal pain after eating, not always though.  Bowel movement is normal, no fever, I feel some gas in stomach and abdominal pain/ stomach burning when lying down on my stomach.  Felt nausea a couple of days but it went away.  If I touch my belly hard it huryts (slightly to the right, not lower side as towards the appendix nor upper right as towards the gallbladder).  Dr prescribed dexlansoprazole which seems to work but still hurts.. Three weeks ago I had upper stomach pain and nausea . Went to the clinic and they said I had a stomach. For the last three weeks, I have still had irritating stomach pain more after eating, nausea and green diarrhea . Also bad upper stomach rumbling and gas. I have bad acid reflux and had my gallbladder Removed 8 years ago after the birth of my son. It almost feels like the attacks I had when I had my gallbladder. Ive been having stomach